In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Some StackOverflow magic to enable importing of local module
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from multiLevelCoSurrogates.local import base_dir

In [ ]:
df = pd.read_csv(base_dir+'records.csv', index_col='index')
df.head()

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(12,9))

grouped = df.groupby(by=['which_model', 'fidelity'])

for ax, group in zip(axes.flatten(), grouped):
    name, sub_df = group
    sub_df.plot('iteration', ['mse_high', 'mse_low', 'mse_diff'], ax=ax)
    ax.set_title('acq on {}, fidelity {} updated'.format(*name))
    
plt.show()